In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from Grocery_3 import Grocery_3

In [3]:
# здесь данные только за 2017 год. Ноутбук с предобработкой я случайно удалила(
# здесть тренировочные данные + праздники в бинарном виде за исключением переносов
# + стоимость топлива + характеристики товара
# в общем, пребобработала по минимому - датасет гигантский и все работает очень медленно
# весь 2017 мои скудные ресурсы тоже не осилили, пришлось откусить 2 месяца и предсказывать
# только последние 2 недели
train = pd.read_csv('final_train.csv')

In [4]:
train['date'] = pd.to_datetime(train['date'])

In [5]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,is_event,family,class,perishable,dcoilwtico
0,2017-01-01,25,99197,1.0,0,1,12,43,0,53.19
1,2017-01-01,25,103665,7.0,0,1,5,184,1,53.19
2,2017-01-01,25,105574,1.0,0,1,12,30,0,53.19
3,2017-01-01,25,105857,4.0,0,1,12,62,0,53.19
4,2017-01-01,25,106716,2.0,0,1,12,19,0,53.19


In [6]:
class RNN(nn.Module):
    
    def __init__(self, num_features, hidden_dim, num_layers=1):
        super().__init__()
        self.num_features = num_features
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size=self.num_features,
                      hidden_size=self.hidden_dim,
                      batch_first=True,
                      num_layers=self.num_layers)
        self.lin = nn.Linear(in_features=self.hidden_dim, 
                            out_features=1)
        
    def forward(self, x):
        #h0 = torch.zeros(self.num_layers,
                         #x.shape[0],
                         #self.hidden_dim).requires_grad_()
        #c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_dim).requires_grad_()
        out, ht= self.rnn(x)
        out = self.lin(out[:, -1])
        return out
    

In [7]:
def train_model(data_loader, model, loss_function, optimizer, epochs_num, batch_len):
    for epoch in tqdm(range(epochs_num)):
        
        num_batches = len(data_loader)
        total_loss = 0
        model.train()
        model.double()
        for ind, (X, y) in enumerate(data_loader):
            output = model(X.double())
            loss = loss_function(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            if ind % batch_len == 0:
                current_loss, current_len = loss.item() ,ind * len(X)
                print(f'Batch_loss: {current_loss}, num_butches {ind} / {num_batches} ')

        avg_loss = total_loss / num_batches     
        print(f"Epoch: {epoch + 1} Train loss: {avg_loss}")
        
        
#         model.eval()
#         loss_accumed = 0
#         for X, y in train_loader:
#             output = model(X.double())
#             loss = loss_function(output, y)
#             loss_accumed += loss.item()
#         eval_loss = loss_accumed / num_batches
#         print(f"Epoch: {epoch + 1} Test_loss: {eval_loss}")

In [ ]:
class Grocery_3():
    
    def __init__(self, dataset, target, scalar):
        self.dataset= dataset
        self.target = target
        self.scalar = scalar()
        self.y = torch.tensor(self.dataset[self.target].to_numpy(dtype='float64')).unsqueeze(1)     
        self.X = self.scalar.fit_transform(torch.tensor(self.dataset.drop(self.target, axis=1).to_numpy(dtype='float64')))
       
        
    def __len__(self):
        return len(self.dataset)
    
    
    def __getitem__(self, i): 
        return self.X[i], self.y[i]

In [8]:
cut_train = train.loc[train['date'] > '2017-05-31']

In [9]:
len(cut_train)

8019163

In [10]:
train.date.max()

Timestamp('2017-08-15 00:00:00')

In [11]:
test_len = len(train.loc[train['date'] > '2017-07-31'])

In [12]:
border = len(cut_train) - test_len

In [13]:
f_train_3 = Grocery_3(cut_train[:border],'unit_sales', StandardScaler)
f_test_3 = Grocery_3(cut_train[border:],'unit_sales', StandardScaler)

In [14]:
train_loader_3 = torch.utils.data.DataLoader(f_train_3,
                          batch_size=1000,
                          shuffle=True,
                          num_workers=2,
                          drop_last=True)
test_loader_3 = torch.utils.data.DataLoader(f_test_3,
                          batch_size=1000,
                          shuffle=False,
                          num_workers=1, 
                          drop_last=True)

In [15]:
# если не поставить размер скрытого слоя равным размеру батча, вылетаю с ошибкой
# несовпадения размерности. Так должно быть или я что-то упустила?
model_3 = RNN(9, 1000)
loss_3 = nn.MSELoss()
optimizer_3 = torch.optim.Adam(model_3.parameters())

In [16]:
# Здесь я забыла функцию доделать и передать тестовые данные,
# повезло, что модель обучилась - нашла нужный код в документации и прогнала тест
# очень странно на 1000 и 4000 батчах ошибка отличается. Почему так происходит?
train_model(train_loader_3, model_3, loss_3, optimizer_3, 1, 1000)

  0%|                                                     | 0/1 [00:00<?, ?it/s]/Users/Viktoryia/Documents/gb/env/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Batch_loss: 243.08438164875133, num_butches 0 / 6448 
Batch_loss: 1469.086419240755, num_butches 1000 / 6448 
Batch_loss: 207.0904727225291, num_butches 2000 / 6448 
Batch_loss: 139.9344296355253, num_butches 3000 / 6448 
Batch_loss: 2848.459923717265, num_butches 4000 / 6448 
Batch_loss: 188.6566576544406, num_butches 5000 / 6448 
Batch_loss: 366.4992351166771, num_butches 6000 / 6448 


  0%|                                                   | 0/1 [7:41:14<?, ?it/s]

Epoch: 1 Train loss: 638.5473600399507


NameError: name 'train_loader' is not defined

In [18]:
torch.save(model_3,'favorita-grocery-sales-forecasting')

In [20]:
model = torch.load('favorita-grocery-sales-forecasting')

In [21]:
print(model)

RNN(
  (rnn): RNN(9, 1000, batch_first=True)
  (lin): Linear(in_features=1000, out_features=1, bias=True)
)


In [22]:
def test_model(data_loader, model, loss_function):

    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")

In [24]:
# Ошибка гигантская. Но даже эти дважды усеченные данные обрабатывались 7 часов
test_model(test_loader_3, model, loss_3)

Test loss: 489.7351021578345


In [ ]:
# справедливости ради - я пыталасть запустить весь 2017 в колабе
# но после 11 часов обработки колаб отвалился, оставив на память 
# вот этот вывод (модель та-же). Ошибка ниже и вроде так не скачет,
# но проверить на тесте увы не могу(


#   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
#   return F.mse_loss(input, target, reduction=self.reduction)
# Batch_loss: 199.00224025591166, num_butches 0 / 20000 
# Batch_loss: 210.78321136725057, num_butches 1000 / 20000 
# Batch_loss: 198.7241058906085, num_butches 2000 / 20000 
# Batch_loss: 145.0831561674167, num_butches 3000 / 20000 
# Batch_loss: 183.056478043171, num_butches 4000 / 20000 
# Batch_loss: 267.8405132705958, num_butches 5000 / 20000 
# Batch_loss: 255.53774859960197, num_butches 6000 / 20000 
# Batch_loss: 221.25796303932802, num_butches 7000 / 20000 